#### Import needed packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from googletrans import Translator

#### load in Facebook, Twitter, Youtube Post/Video Performance file

In [3]:
df_twit = pd.read_excel('Tweet text.xlsx')
df_fb = pd.read_excel('Facebook Text.xlsx')
df_yt = pd.read_excel('Youtube Text.xlsx')

In [32]:
df_twit.head(0)

,Tweet id,Tweet permalink,Tweet text,time,impressions,engagements,engagement rate,retweets,replies,likes,...,permalink clicks,app opens,app installs,follows,email tweet,dial phone,media views,media engagements,year,month


In [33]:
df_twit.tail(0)

,Tweet id,Tweet permalink,Tweet text,time,impressions,engagements,engagement rate,retweets,replies,likes,...,permalink clicks,app opens,app installs,follows,email tweet,dial phone,media views,media engagements,year,month


In [34]:
df_fb.head(0)

,Permalink,Post Message,Type,Posted,Lifetime Post Total Reach,Lifetime Post organic reach,Lifetime Post Paid Reach,Lifetime Post Total Impressions,Lifetime Post Organic Impressions,Lifetime Post Paid Impressions,...,Lifetime Post Paid Impressions by people who have liked your Page,Lifetime Paid reach of a post by people who like your Page,Lifetime People who have liked your Page and engaged with your post,like,share,comment,other clicks,link clicks,photo view,Month


In [35]:
df_fb.tail(0)

,Permalink,Post Message,Type,Posted,Lifetime Post Total Reach,Lifetime Post organic reach,Lifetime Post Paid Reach,Lifetime Post Total Impressions,Lifetime Post Organic Impressions,Lifetime Post Paid Impressions,...,Lifetime Post Paid Impressions by people who have liked your Page,Lifetime Paid reach of a post by people who like your Page,Lifetime People who have liked your Page and engaged with your post,like,share,comment,other clicks,link clicks,photo view,Month


In [36]:
df_yt.head(0)

,video,video_title,watch_time_minutes,views,subscribers,video_thumbnail_impressions,video_thumbnail_impressions_ctr


In [37]:
df_yt.tail(1)

,video,video_title,watch_time_minutes,views,subscribers,video_thumbnail_impressions,video_thumbnail_impressions_ctr
29,Total,NaN,36270.1927,19790,103,NaN,NaN


#### Function 1: Topic Modelling for All Texts

In [28]:
def topic_model(file, colname, topics_num):
    
    import nltk
    import pandas as pd
    from nltk.corpus import stopwords
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.decomposition import TruncatedSVD

    if 'csv' in file:
        df = pd.read_csv(file)
    elif 'xlsx' in file:
        df = pd.read_excel(file)
    
    # Clean the text
    df = df.dropna(subset=[colname])
    df['clean_title'] = df[colname].str.replace("[^a-zA-Z#]", " ")
    df['clean_title'] = df['clean_title'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
    df['clean_title'] = df['clean_title'].apply(lambda x: x.lower())
    
    # deal with the stop word
    stop_words = set(stopwords.words('english'))
    tokenized_doc = df['clean_title'].apply(lambda x: x.split())
    tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
    
    # merge the tokenized word back to sentences again
    detokenized_doc = []
    for i in range(len(df)):
        t = ' '.join(tokenized_doc[i])
        detokenized_doc.append(t)
    df['clean_title'] = detokenized_doc

    # vectorize it
    vectorizer = TfidfVectorizer(max_features= 500, # keep top 500 terms 
                                 max_df = 0.5, 
                                 smooth_idf=True)

    X = vectorizer.fit_transform(df['clean_title'])
    

    # SVD represent documents and terms in vectors 
    svd_model = TruncatedSVD(n_components = topics_num, algorithm='randomized', n_iter=100, random_state=122)
    svd_model.fit(X)
    
    terms = vectorizer.get_feature_names()

    for i, comp in enumerate(svd_model.components_):
        terms_comp = zip(terms, comp)
        sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:6]
        print("Topic "+str(i)+": ")
        print('-------')
        for t in sorted_terms:
            print(t[0])
            print(' ')

#### Function 2: Topic Modelling for All Texts

In [27]:
def topic_model_quantile(file, colname, metric_col, lower_quantile_no, upper_quantile_no ,topics_num):
    
    import nltk
    import pandas as pd
    from nltk.corpus import stopwords
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.decomposition import TruncatedSVD

    if 'csv' in file:
        df = pd.read_csv(file)
    elif 'xlsx' in file:
        df = pd.read_excel(file)
    
    df = df.dropna(subset=[colname])
    
    # Subset the data with quantile
    lower_quantile, upper_quantile = df[metric_col].quantile([lower_quantile_no/100, upper_quantile_no/100])
    df = df.loc[(df[metric_col] > lower_quantile) & (df[metric_col] < upper_quantile)]
    
    df.reset_index(inplace=True)

    # Clean the text
    df['clean_title'] = df[colname].str.replace("[^a-zA-Z#]", " ")
    df['clean_title'] = df['clean_title'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
    df['clean_title'] = df['clean_title'].apply(lambda x: x.lower())
    
    
    stop_words = set(stopwords.words('english'))

    tokenized_doc = df['clean_title'].apply(lambda x: x.split())

    tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
    detokenized_doc = []
    for i in range(len(df)):
        t = ' '.join(tokenized_doc[i])
        detokenized_doc.append(t)

    df['clean_title'] = detokenized_doc

    vectorizer = TfidfVectorizer(max_features= 500, # keep top 1000 terms 
                                 max_df = 0.5, 
                                 smooth_idf=True)

    X = vectorizer.fit_transform(df['clean_title'])
    

    # SVD represent documents and terms in vectors 
    svd_model = TruncatedSVD(n_components = topics_num, algorithm='randomized', n_iter=100, random_state=122)

    svd_model.fit(X)
    
    terms = vectorizer.get_feature_names()

    for i, comp in enumerate(svd_model.components_):
        terms_comp = zip(terms, comp)
        sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:]
        print("Topic "+str(i)+": ")
        print('-------')
        for t in sorted_terms:
            print(t[0])
            print(' ')

In [26]:
topic_model('Youtube Text.xlsx','video_title',3)

Topic 0: 
-------
injuries
 
needlestick
 
preventing
 
proper
 
farms
 
Topic 1: 
-------
agujas
 
apropiado
 
lesiones
 
piquetes
 
previniendo
 
Topic 2: 
-------
health
 
safety
 
center
 
midwest
 
umash
 


In [21]:
topic_model('Tweet text.xlsx', 'Tweet text',5)

Topic 0: 
safety
 
farm
 
health
 
check
 
safe
 
Topic 1: 
safe
 
thanks
 
farm
 
stay
 
sharing
 
Topic 2: 
thanks
 
safe
 
sharing
 
health
 
stay
 
Topic 3: 
usagcenters
 
nfshw
 
safety
 
thanks
 
necasag
 
Topic 4: 
usagcenters
 
webinar
 
work
 
asap
 
agriculture
 


In [29]:
topic_model_quantile('Tweet text.xlsx', 'Tweet text', 'engagements', 75, 100 , 3)

Topic 0: 
-------
safety
 
farm
 
check
 
farmsafety
 
stress
 
month
 
Topic 1: 
-------
stress
 
women
 
resiliency
 
agriculture
 
find
 
cultivating
 
Topic 2: 
-------
grain
 
week
 
safety
 
rescue
 
women
 
open
 


In [31]:
topic_model_quantile('Facebook Text.xlsx', 'Post Message', 'Lifetime Post Total Impressions', 0, 40, 4)

Topic 0: 
-------
safety
 
health
 
upper
 
center
 
midwest
 
agricultural
 
Topic 1: 
-------
farm
 
umash
 
http
 
partner
 
check
 
safety
 
Topic 2: 
-------
umash
 
field
 
catch
 
partner
 
activities
 
conference
 
Topic 3: 
-------
umash
 
field
 
http
 
great
 
news
 
story
 
